In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv
/kaggle/input/en-core-web-md/en_core_web_md/meta.json
/kaggle/input/en-core-web-md/en_core_web_md/__init__.py
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/tokenizer
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/LICENSES_SOURCES
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/LICENSE
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/README.md
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/meta.json
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/accuracy.json
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/config.cfg
/kag

In [2]:
import warnings
warnings.simplefilter("ignore", ResourceWarning)

## Let's load the training data

In [3]:
summary_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompt_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')

## Check dataframe description

In [4]:
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7165 entries, 0 to 7164
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   student_id  7165 non-null   object 
 1   prompt_id   7165 non-null   object 
 2   text        7165 non-null   object 
 3   content     7165 non-null   float64
 4   wording     7165 non-null   float64
dtypes: float64(2), object(3)
memory usage: 280.0+ KB


In [5]:
prompt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prompt_id        4 non-null      object
 1   prompt_question  4 non-null      object
 2   prompt_title     4 non-null      object
 3   prompt_text      4 non-null      object
dtypes: object(4)
memory usage: 256.0+ bytes


## Primary key to combine dataframe on

In [6]:
prompt_df['prompt_id'].unique()

array(['39c16e', '3b9047', '814d6b', 'ebad26'], dtype=object)

In [7]:
summary_df['prompt_id'].unique()

array(['814d6b', 'ebad26', '3b9047', '39c16e'], dtype=object)

## Merge dataframe based on primary key into one

In [8]:
train_df = summary_df.merge(right=prompt_df,how='inner', on='prompt_id')

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7165 entries, 0 to 7164
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   student_id       7165 non-null   object 
 1   prompt_id        7165 non-null   object 
 2   text             7165 non-null   object 
 3   content          7165 non-null   float64
 4   wording          7165 non-null   float64
 5   prompt_question  7165 non-null   object 
 6   prompt_title     7165 non-null   object 
 7   prompt_text      7165 non-null   object 
dtypes: float64(2), object(6)
memory usage: 503.8+ KB


### Let's perform a little exploratory analysis on the data to see what samples looks like

In [10]:
print('''Question: %s \n 
Title: %s \n
Text: %s'''%(train_df.iloc[1]['prompt_question'],train_df.iloc[1]['prompt_title'],train_df.iloc[1]['prompt_text']))

Question: Summarize how the Third Wave developed over such a short period of time and why the experiment was ended. 
 
Title: The Third Wave 

Text: Background 
The Third Wave experiment took place at Cubberley High School in Palo Alto, California during the first week of April 1967. History teacher Ron Jones, finding himself unable to explain to his students how people throughout history followed the crowd even when terrible things were happening, decided to demonstrate it to his students through an experiment. Jones announced that he was starting a movement aimed to eliminate democracy. Jones named the movement “The Third Wave” as a symbol of strength, referring to the mythical belief that the third in a series of waves is the strongest. One of the central points of this movement was that democracy’s main weakness is that it favors the individual over the whole community. Jones emphasized this main point of the movement when he created this catchy motto: “Strength through discipline,

### Let's check what the student summary looks like

In [11]:
from pprint import pformat
print(pformat('''Question: %s
Summary: %s
Content Score: %s
Wording Score: %s'''%(train_df.iloc[0]['prompt_question'],train_df.iloc[0]['text'], train_df.iloc[0]['content'],
                           train_df.iloc[0]['wording'])))

('Question: Summarize how the Third Wave developed over such a short period of '
 'time and why the experiment was ended.\n'
 'Summary: The third wave was an experimentto see how people reacted to a new '
 'one leader government. It gained popularity as people wanted to try new '
 'things. The students follow anything that is said and start turning on '
 'eachother to gain higher power. They had to stop the experement as too many '
 'people got to radical with it blindly following there leader\n'
 'Content Score: 0.205682506482641\n'
 'Wording Score: 0.380537638762288')


## Let's build a helper function that displays N text in a pretty format

In [12]:
def display_text(s=1,n=10,df=train_df):
    """
    This function takes n as input and displays n section of text
    """
    for i in range(s,n):
        p_sum = pformat('''Question: %s \nSummary: %s \nContent Score: %s \nWording Score: %s'''%(df.iloc[i]['prompt_question'],df.iloc[i]['text'], df.iloc[i]['content'],
                           df.iloc[i]['wording']))
        print(p_sum,end='\n\n')

In [13]:
display_text(s=200,n=220,df=train_df)

('Question: Summarize how the Third Wave developed over such a short period of '
 'time and why the experiment was ended. \n'
 'Summary: The Third Wave developed over such a short period of time. Jones '
 'emphasized many simple rules and made sure the students did it right. '
 'Students also had to salute each other in a specific way that the teacher '
 'had made. On the fourth day, Jones claimed that things started getting out '
 'of control because the students became increasingly involved in the project, '
 'so their discipline and loyalty to the project was overly outstanding. So '
 'Jones decided to terminate the movement and lied to the students by '
 'announcing that the Third Wave was a part of a nationwide movement.  \n'
 'Content Score: 1.46743085810556 \n'
 'Wording Score: 1.84017156306968')

('Question: Summarize how the Third Wave developed over such a short period of '
 'time and why the experiment was ended. \n'
 'Summary: It happened so fastly because the orig nail kid

#### Combine the Title, Question, Context and Summary text into one large block of text

In [14]:
sample = 'Title:\n'+train_df.iloc[0]['prompt_title']+'\nQuestion:\n'+train_df.iloc[0]['prompt_question']+'\nText:\n'+train_df.iloc[0]['prompt_text']+'\nAnswer:\n'+train_df.iloc[0]['text']

In [15]:
print(sample)

Title:
The Third Wave
Question:
Summarize how the Third Wave developed over such a short period of time and why the experiment was ended.
Text:
Background 
The Third Wave experiment took place at Cubberley High School in Palo Alto, California during the first week of April 1967. History teacher Ron Jones, finding himself unable to explain to his students how people throughout history followed the crowd even when terrible things were happening, decided to demonstrate it to his students through an experiment. Jones announced that he was starting a movement aimed to eliminate democracy. Jones named the movement “The Third Wave” as a symbol of strength, referring to the mythical belief that the third in a series of waves is the strongest. One of the central points of this movement was that democracy’s main weakness is that it favors the individual over the whole community. Jones emphasized this main point of the movement when he created this catchy motto: “Strength through discipline, stre

In [16]:
train_df['full_text'] = train_df.apply(lambda x: 'Title:\n'+x['prompt_title']+
               '\nQuestion:\n'+x['prompt_question']+
               '\nText:\n'+x['prompt_text']+
               '\nAnswer:\n'+x['text'],axis=1)

### Let's perform text preprocessing on our df full_text column

In [17]:
import warnings
warnings.simplefilter("ignore", UserWarning)

### Input all libraries

In [18]:
import spacy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV

In [19]:
from string import punctuation as punc
from collections import defaultdict
import spacy
from string import punctuation as punc
from tqdm import tqdm

nlp = spacy.load("/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/")

def process_text(text):
    doc = nlp(text)
    lemmas = []
    stops = 0
    pos = defaultdict(int,{'VERB': 0, "NOUN": 0})
    answer_length = len(text)

    for token in doc:
        if token.is_stop:
            stops += 1
        else:
            if token.pos_ == "VERB":
                pos[token.pos_] += 1
            elif token.pos_ == "NOUN":
                pos[token.pos_] += 1
        lemmas.append(token.lemma_.lower())
    pos = dict(pos)
    return ' '.join(lemmas), stops, pos['VERB'], pos['NOUN'], answer_length

def count_punc(text):
    return len([p for p in text if p in punc])

# Overlap Score
def overlap(sample):
    answer = set(sample['text'].lower().split())
    context = set(sample['prompt_text'].lower().split())
    intersection = len(context.intersection(answer))
    difference = len(context.difference(answer))
    union = len(context.union(answer))
    overlap_score = intersection/float(len(context))
    jaccard_similarity = intersection/union
    return intersection, difference, union, overlap_score, jaccard_similarity


In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances


def preprocess_data(summary_df, prompt_df, ngram_range=(1,1)):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')

    train_df = train_df.assign(
    full_text='Title:\n' + train_df['prompt_title'] +
            '\nQuestion:\n' + train_df['prompt_question'] +
            '\nText:\n' + train_df['prompt_text']
    )

    print('processing text data')

    # Refactored text processing
    def process_texts(column_name):
        return zip(*train_df[column_name].apply(process_text).tolist())

    train_df['intersection'], train_df['difference'], train_df['union'], train_df['overlap_score'], train_df['jaccard_similarity'] = zip(*train_df.apply(overlap, axis=1).tolist())

    train_df['lemmas'],train_df['summary_stopwords'],train_df['summary_verbs'], train_df['summary_nouns'], train_df['length_of_summary'] = process_texts('text')
    train_df['full_text'], train_df['full_text_stopwords'],train_df['full_text_verbs'], train_df['full_text_nouns'], train_df['length_of_full_text'] = process_texts('full_text')

    print('processing features.....')
    # Process other columns
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)

    cols = ['length_of_summary', 'summary_punctuation','summary_stopwords','summary_verbs','summary_nouns',
      'intersection','difference','union', 'overlap_score', 'jaccard_similarity',
      'full_text_stopwords','full_text_verbs','full_text_nouns','length_of_full_text','content', 'wording']

    print('applying vectorizer.....')
    global vectorizer, pca
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    transformer = TfidfTransformer(sublinear_tf=True)
    transformed_df = transformer.fit_transform(vectorizer.fit_transform(train_df['full_text']))
    summary_transformed_df = transformer.fit_transform(vectorizer.transform(train_df['lemmas']))

    print('scaling features.....')
    scaler = StandardScaler(with_mean=True, with_std=True,)
    X_sample_scaled = scaler.fit_transform(transformed_df.toarray())
    X_summary_scaled = scaler.fit_transform(summary_transformed_df.toarray())

    print('decomposing features.......')
    components = round(summary_transformed_df.shape[1] * (43.6/100))
    pca = TruncatedSVD(n_components=components,algorithm='arpack', random_state=42)
    pca_bag = pca.fit_transform(X_sample_scaled)
    summary_bag = pca.fit_transform(X_summary_scaled)
    similarity = np.zeros((train_df.shape[0], 2))
    for i, data in train_df.iterrows():
        full_bags = transformer.transform(vectorizer.transform([data['full_text']]))
        summary_bags = transformer.transform(vectorizer.transform([data['lemmas']]))
        similarity[i, 0] = cosine_similarity(full_bags, summary_bags)[0][0]
        similarity[i, 1] = euclidean_distances(full_bags, summary_bags)[0][0]

    stacked_bag = np.hstack((pca_bag, summary_bag,similarity,train_df[cols].values))

    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return stacked_bag, student_ids

In [21]:
import time

start_time = time.time()

bag_of_words, student_ids = preprocess_data(summary_df,prompt_df,ngram_range=(1,1))

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Preprocessed function executed in: {elapsed_time:.2f} seconds")

processing text data
processing features.....
applying vectorizer.....
scaling features.....
decomposing features.......
collecting outputs.........
Preprocessed function executed in: 810.81 seconds


### processed summary - 
```
similarity, euclidean,full_text, summary_text length_of_summary summary_punctuation summary_stopwords summary_verbs','summary_nouns','intersection',
'difference','union'overlap_score','jaccard_similarity','full_text_stopwords','full_text_verbs','full_text_nouns','length_of_full_text'          
```

In [22]:
bag_of_words.shape

(7165, 764)

In [23]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (7,5)

In [24]:
def get_plot(d_lower,d_higher,cumsum):
    fig, axis = plt.subplots()
    axis.step(range(len(cumsum)),cumsum, where='mid', c='b', label='Explained Ration')
    axis.vlines(d_higher, 0, cumsum[d_higher], colors='orange', linestyles='--',label=f'{d_higher} components')
    axis.vlines(d_lower, 0, cumsum[d_lower],colors='red', linestyles='--', label=f'{d_lower} components')
    axis.hlines(cumsum[d_higher], -200, d_higher, colors='orange', linestyles='--', label=f'{cumsum[d_higher]:.2f} cumsum')
    axis.hlines(cumsum[d_lower], -200, d_lower,colors='red', linestyles='--', label=f'{cumsum[d_lower]:.2f} cumsum')
    axis.set_xlim([-200, 5000])
    axis.set_xlabel('Principal Components Index')
    axis.set_ylabel('Explained Variance Ratio')
    axis.grid(True)
    plt.legend(loc='best')
    plt.show()

In [25]:
# get_plot(d_lower=d_95,d_higher=d_99,cumsum=cumsum)

In [26]:
X_train,X_test,y_train,y_test = train_test_split(bag_of_words[:,:-2] ,bag_of_words[:,-2:],test_size=0.1,shuffle=True,random_state=11)

In [27]:
from warnings import filterwarnings
filterwarnings(action='always')

In [28]:
def mcrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred)**2, axis=0)))

In [29]:
model_rf = MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=11,
                                                     max_features=0.6,
                                                     min_samples_split=10,
                                                     n_estimators=400,
                                                     n_jobs=-1,
                                                     random_state=42))

In [30]:
model_rf.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=11,
                                                     max_features=0.6,
                                                     min_samples_split=10,
                                                     n_estimators=400,
                                                     n_jobs=-1,
                                                     random_state=42))

In [31]:
y_preds = model_rf.predict(X_test)
mcrmse(y_test,y_preds)

0.51641961787614

In [32]:
X_train.shape

(6448, 762)

In [33]:
model_rf.score(X_train, y_train)

0.899528369403503

In [34]:
model_rf = MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=11,
                                                     max_features=0.6,
                                                     min_samples_split=10,
                                                     n_estimators=400,
                                                     n_jobs=-1,
                                                     random_state=42))
model_rf.fit(bag_of_words[:,:-2] ,bag_of_words[:,-2:])

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=11,
                                                     max_features=0.6,
                                                     min_samples_split=10,
                                                     n_estimators=400,
                                                     n_jobs=-1,
                                                     random_state=42))

In [35]:
model_rf.score(bag_of_words[:,:-2], bag_of_words[:,-2:])

0.896217387141427

In [36]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def preprocess_data(test_summary, test_prompts, vectorizer=vectorizer):
    train_df = test_summary.merge(right=test_prompts, how='inner', on='prompt_id')

    train_df = train_df.assign(
    full_text='Title:\n' + train_df['prompt_title'] +
            '\nQuestion:\n' + train_df['prompt_question'] +
            '\nText:\n' + train_df['prompt_text']
    )

    print('processing text data')

    # Refactored text processing
    def process_texts(column_name):
        return zip(*train_df[column_name].apply(process_text).tolist())

    train_df['intersection'], train_df['difference'], train_df['union'], train_df['overlap_score'], train_df['jaccard_similarity'] = zip(*train_df.apply(overlap, axis=1).tolist())

    train_df['lemmas'],train_df['summary_stopwords'],train_df['summary_verbs'], train_df['summary_nouns'], train_df['length_of_summary'] = process_texts('text')
    train_df['full_text'], train_df['full_text_stopwords'],train_df['full_text_verbs'], train_df['full_text_nouns'], train_df['length_of_full_text'] = process_texts('full_text')

    print('processing features.....')
    # Process other columns
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)

    cols = ['length_of_summary', 'summary_punctuation','summary_stopwords','summary_verbs','summary_nouns',
      'intersection','difference','union', 'overlap_score', 'jaccard_similarity',
      'full_text_stopwords','full_text_verbs','full_text_nouns','length_of_full_text']

    print('applying vectorizer.....')
    transformer = TfidfTransformer(sublinear_tf=True)
    transformed_df = transformer.fit_transform(vectorizer.transform(train_df['full_text']))
    summary_transformed_df = transformer.fit_transform(vectorizer.transform(train_df['lemmas']))

    print('scaling features.....')
    scaler = StandardScaler(with_mean=True, with_std=True,)
    X_sample_scaled = scaler.fit_transform(transformed_df.toarray())
    X_summary_scaled = scaler.fit_transform(summary_transformed_df.toarray())

    print('decomposing features.......')
    pca_bag = pca.transform(X_sample_scaled)
    summary_bag = pca.transform(X_summary_scaled)
    similarity = np.zeros((train_df.shape[0], 2))
    for i, data in train_df.iterrows():
        full_bags = transformer.transform(vectorizer.transform([data['full_text']]))
        summary_bags = transformer.transform(vectorizer.transform([data['lemmas']]))
        similarity[i, 0] = cosine_similarity(full_bags, summary_bags)[0][0]
        similarity[i, 1] = euclidean_distances(full_bags, summary_bags)[0][0]

    stacked_bag = np.hstack((pca_bag, summary_bag,similarity,train_df[cols].values))

    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return stacked_bag, student_ids

In [37]:
test_summary = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
test_prompts = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')

In [38]:
test_df,ids = preprocess_data(test_summary, test_prompts, vectorizer=vectorizer)

processing text data
processing features.....
applying vectorizer.....
scaling features.....
decomposing features.......
collecting outputs.........


In [39]:
y_preds = model_rf.predict(test_df)

In [40]:
df_test = pd.DataFrame(y_preds,columns=['content','wording'],index=ids).reset_index()
df_test = df_test.rename({'index':'student_id'},axis=1)

In [41]:
df_test.to_csv("submission.csv", index=False)